Neural Network Regression (attempt) with Pytorch

In [1]:
# imports
import torch
import torch.nn as nn
# import torchaudio --> dunno what this is for
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# adding the data utils 
from data_utils import *

# Ignore warnings >:D
import warnings
warnings.filterwarnings("ignore")
print('sanity')

sanity


# Dealing with the data (for testing the NN initally)

In [2]:
# loading data from any given .csv
data_850 = pd.read_csv('data_table_NA850_nonfiltered_updated.csv')

# for now, drop the Time, Lat, and Lon Data -- will revisit later
data_850 = data_850.drop(columns = ['Time','Lat','Lon'])

data_850.head()

# len(data_850)

print(type(data_850.loc[1][1]))

<class 'numpy.float64'>


In [3]:
# Maybe not NEEDED but is good PyTorch practice: basically a class that inherits from pytorch's Dataset class
# Have to override the init, getitem, and len functions
# this will be easy to modify for different numbers of features


class CustomWaveData(Dataset):
    def __init__(self, dataframe):
        self.y = torch.tensor(dataframe['LH'].values,dtype=torch.float32)
        self.x = torch.tensor(dataframe.drop(columns = ['LH']).values,dtype=torch.float32)
        self.size = len(dataframe)
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.size
    
    

In [4]:
# # create the data class with our custom data
# data = CustomWaveData(data_850)

In [5]:
# dataloaders
# set batch_size = 64 for now, can play around with this value when we use more data
# dataloader = DataLoader(dataset = data, batch_size = 64, shuffle = True)

# The REAL Data

In [6]:
# I am not good at naming things...
real_data = pd.read_csv('NA850data_2000-2014.csv')
real_data.head()

,AirTemp,QV,Omega,SeaLevPress,UWinds,VWinds,LH,Lat,Lon,PtIndex,WaveTrajectory,Time
0,7.060846,7.267001,0.000642,1021.565625,-11.894431,-5.276164,12.888080,25.064459,-35.368896,0,1,2000010100
1,10.865533,9.469151,-0.000444,1015.694375,-12.986907,-2.199314,16.006929,20.454613,-54.790527,18,1,2000010306
2,7.100183,7.474899,-0.000296,1021.084141,-11.230261,-3.172798,-10.217120,24.480576,-36.771637,1,1,2000010103
3,11.154810,9.294266,0.000126,1015.156875,-11.698672,-2.088208,1.020178,19.468901,-55.690735,19,1,2000010309
4,7.068231,7.335827,0.000323,1020.302969,-10.246526,-1.993664,0.511768,24.649042,-37.882874,2,1,2000010106


In [7]:
# may be a better way of doing this, feel free to fix if so (probably so)
real_data = add_deltas_and_time(real_data)
real_data = add_extreme(real_data)
real_data = scale_data(real_data)
real_data.head()

,AirTemp,QV,Omega,SeaLevPress,UWinds,VWinds,LH,Lat,Lon,PtIndex,...,dOmega,dSeaLevPress,dUWinds,dVWinds,dLH,dLat,dLon,dPtIndex,dWaveTrajectory,Extreme
Time,,,,,,,,,,,,,,,,,,,,,
2000-01-01 00:00:00,-5.894685,-0.921917,0.663327,2.255124,-0.838782,-1.512121,1.682001,1.284264,0.046455,-0.904762,...,0.001439,-0.031599,0.008588,-0.019885,-0.000014,-0.051313,1.086233,-1.0,0.0,0.0
2000-01-03 06:00:00,-3.680371,-0.425460,-0.022623,0.672601,-1.048340,-0.713308,2.162334,0.730487,-0.914611,-0.047619,...,-0.935016,-2.753314,-0.637262,1.909090,1.128003,-9.334594,-31.353357,17.0,0.0,0.0
2000-01-01 03:00:00,-5.871791,-0.875048,0.071067,2.125345,-0.711381,-0.966044,-1.876418,1.214122,-0.022959,-0.857143,...,0.129345,2.466917,1.047082,-0.630193,-9.484652,8.056147,31.182853,-18.0,0.0,1.0
2000-01-03 09:00:00,-3.512013,-0.464887,0.337309,0.527725,-0.801232,-0.684463,-0.145768,0.612075,-0.959157,0.000000,...,0.364911,-2.779281,-0.268328,0.660079,4.064260,-10.143794,-30.513986,17.0,0.0,0.0
2000-01-01 06:00:00,-5.890387,-0.906401,0.462108,1.914790,-0.522681,-0.659917,-0.224068,1.234360,-0.077947,-0.809524,...,0.171814,2.353958,0.867068,0.039388,-0.183894,10.380425,30.830375,-18.0,0.0,0.0


In [8]:
# for now, drop the Time, Lat, and Lon Data -- will revisit later
# for testing, trying out only the strongest features
real_data = real_data.drop(columns = ['SeaLevPress', 'UWinds', 'PtIndex', 'dOmega', 'dUWinds', 'dWaveTrajectory', 'dAirTemp', 'dQV', 'dSeaLevPress', 'dVWinds'
                                     , 'dLH', 'dLat', 'dLon', 'dPtIndex', 'Extreme', 'WaveTrajectory'])

# I think only keeping the values found to be the most important should be kept here for testing
# omega, qv, vwinds, airtemp

In [9]:
real_data

,AirTemp,QV,Omega,VWinds,LH,Lat,Lon
Time,,,,,,,
2000-01-01 00:00:00,-5.894685,-0.921917,0.663327,-1.512121,1.682001,1.284264,0.046455
2000-01-03 06:00:00,-3.680371,-0.425460,-0.022623,-0.713308,2.162334,0.730487,-0.914611
2000-01-01 03:00:00,-5.871791,-0.875048,0.071067,-0.966044,-1.876418,1.214122,-0.022959
2000-01-03 09:00:00,-3.512013,-0.464887,0.337309,-0.684463,-0.145768,0.612075,-0.959157
2000-01-01 06:00:00,-5.890387,-0.906401,0.462108,-0.659917,-0.224068,1.234360,-0.077947
...,...,...,...,...,...,...,...
2014-12-13 21:00:00,-1.870051,-0.470857,0.865670,0.123725,0.932623,1.096264,-0.615757
2014-12-14 00:00:00,-2.023152,-0.592185,-0.263638,0.745227,0.512284,1.096267,-0.629184
2014-12-14 03:00:00,-1.806093,-0.416218,0.708105,0.656077,0.344789,1.082288,-0.640881


In [10]:
# create the data class with our custom data
data = CustomWaveData(real_data)
dataloader = DataLoader(dataset = data, batch_size = 512, shuffle = True)

In [11]:
print(len(real_data))

20304


# Defining the Model

In [12]:
# the function linear_relu_stack is essentially the whole network (in a sequential layer)
# we can play with the number and size of layers


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # initialize the sequential/feedforward net here
        self.linear_relu_stack = nn.Sequential(
        # input layer, 6 inputs and 6 outputs for now
        nn.Linear(6,8),
        # relu activation
        nn.ReLU(),
        
        # update the numbers for input -> output for a given layer    
            
        # hidden layer 1
        nn.Linear(8,8),
        nn.ReLU(),
            
        # hidden layer 2
        nn.Linear(8,8),
        nn.ReLU(),
            
        # output -> we can always play around with the shape of the network
        # I just want to see that this works for now
        nn.Linear(8,1)
        )
        
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits
    

    

In [13]:
# viewing the model and testing once

model = NeuralNetwork()
# model = model.float()
print(model)
test_data = data[0]
X,y = test_data
print(X)
test_res = model(X)
print(test_res)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=6, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=8, bias=True)
    (5): ReLU()
    (6): Linear(in_features=8, out_features=1, bias=True)
  )
)
tensor([-5.8947, -0.9219,  0.6633, -1.5121,  1.2843,  0.0465])
tensor([0.1995], grad_fn=<AddBackward0>)


# Loss and Optimization 

In [14]:
# define the loss we use: regression = MSE
loss_fn = nn.MSELoss()

# define the optimization: Stochastic Gradient Descent, lr = learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# Train and Test Loops

In [15]:
# set the number of epochs -- can and should change for testing
# epochs = 50

In [16]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        # print(y)
        # print(pred)
        loss = loss_fn(pred,y)
        print(loss)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [17]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
    test_loss /= num_batches
    print(f" Avg loss: {test_loss:>8f} \n")

In [18]:
epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(dataloader, model, loss_fn, optimizer)
    test(dataloader, model, loss_fn)
print("Done!")

# prepare for a wall of text

Epoch 1
-------------------------------
tensor(5.6127, grad_fn=<MseLossBackward0>)
loss: 5.612726  [  512/20304]
tensor(6.9695, grad_fn=<MseLossBackward0>)
tensor(4.7601, grad_fn=<MseLossBackward0>)
tensor(5.1198, grad_fn=<MseLossBackward0>)
tensor(6.2963, grad_fn=<MseLossBackward0>)
tensor(4.5271, grad_fn=<MseLossBackward0>)
tensor(3.8449, grad_fn=<MseLossBackward0>)
tensor(5.6975, grad_fn=<MseLossBackward0>)
tensor(3.6873, grad_fn=<MseLossBackward0>)
tensor(5.2797, grad_fn=<MseLossBackward0>)
tensor(6.4293, grad_fn=<MseLossBackward0>)
tensor(5.6833, grad_fn=<MseLossBackward0>)
tensor(4.3425, grad_fn=<MseLossBackward0>)
tensor(4.5223, grad_fn=<MseLossBackward0>)
tensor(4.6410, grad_fn=<MseLossBackward0>)
tensor(5.7037, grad_fn=<MseLossBackward0>)
tensor(4.5794, grad_fn=<MseLossBackward0>)
tensor(4.0192, grad_fn=<MseLossBackward0>)
tensor(6.4945, grad_fn=<MseLossBackward0>)
tensor(4.2809, grad_fn=<MseLossBackward0>)
tensor(8.3347, grad_fn=<MseLossBackward0>)
tensor(3.4315, grad_fn=<Mse

In [19]:
# testing a prediction

model.eval()
with torch.no_grad():
    test_data = data[0]
    X,y = test_data
    print(X)
    test_res = model(X)
    print(test_res)
    
# for i in range(100):
#     model.eval()
#     with torch.no_grad():
#         test_data = data[i]
#         X,y = test_data
#         print(X)
#         test_res = model(X)
#         print('test res: ' + str(test_res) + ' Actual: ' + y)
#         print(test_res)
#         print( ' Actual: ' + str(y))
    

tensor([-5.8947, -0.9219,  0.6633, -1.5121,  1.2843,  0.0465])
tensor([0.6927])


Now, we should try to see which features could lead to the most optimal model (?)

In [20]:
real_data

,AirTemp,QV,Omega,VWinds,LH,Lat,Lon
Time,,,,,,,
2000-01-01 00:00:00,-5.894685,-0.921917,0.663327,-1.512121,1.682001,1.284264,0.046455
2000-01-03 06:00:00,-3.680371,-0.425460,-0.022623,-0.713308,2.162334,0.730487,-0.914611
2000-01-01 03:00:00,-5.871791,-0.875048,0.071067,-0.966044,-1.876418,1.214122,-0.022959
2000-01-03 09:00:00,-3.512013,-0.464887,0.337309,-0.684463,-0.145768,0.612075,-0.959157
2000-01-01 06:00:00,-5.890387,-0.906401,0.462108,-0.659917,-0.224068,1.234360,-0.077947
...,...,...,...,...,...,...,...
2014-12-13 21:00:00,-1.870051,-0.470857,0.865670,0.123725,0.932623,1.096264,-0.615757
2014-12-14 00:00:00,-2.023152,-0.592185,-0.263638,0.745227,0.512284,1.096267,-0.629184
2014-12-14 03:00:00,-1.806093,-0.416218,0.708105,0.656077,0.344789,1.082288,-0.640881
